In [ ]:
import pandas as pd
from requests import get
import json

In [8]:
from datetime import date

# Create .xlsx template file for tax data parsing

In [ ]:
def create_tax_template(output_path: str = '../data/template.xlsx') -> None:
    """
    Create an Excel template file for tax data parsing.
    
    Args:
        output_path (str): Path where the template file will be saved
    """
    template_df = pd.DataFrame({
        "Transaction amount": [],
        "Currency": [],
        "Transaction date": [],
        "Income source": [],
    })
    template_df.to_excel(output_path, index=False)
    print(f"✅ Template created at: {output_path}")

# Create the template
create_tax_template()

## fill example file

In [ ]:
#

# Parse filled tax data from .xlsx file

In [ ]:
def load_and_process_tax_data(file_path: str, sheet_name: str = 'Data', prev_month_amount: float = 0.0) -> pd.DataFrame:
    """
    Load tax data from Excel file and process it with currency conversion.
    
    Args:
        file_path (str): Path to the Excel file containing tax data
        sheet_name (str): Name of the sheet to read from
        prev_month_amount (float): Previous month's amount in GEL
        
    Returns:
        pd.DataFrame: Processed DataFrame with currency rates and GEL amounts
    """
    print(f"📊 Loading tax data from: {file_path}")
    
    # Load the data
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    print(f"✅ Loaded {len(df)} transactions")
    
    # Convert transaction dates to proper date format if needed
    if 'Transaction date' in df.columns:
        df['Transaction date'] = pd.to_datetime(df['Transaction date']).dt.date
    
    # Get currency rates for each transaction
    print("🔄 Fetching currency rates...")
    df['rate'] = df.apply(lambda row: get_currency_rate(row['Currency'], row['Transaction date']), axis=1)
    
    # Calculate amounts in GEL
    df['amount_in_gel'] = df['Transaction amount'] * df['rate']
    
    # Calculate year-to-date total
    current_month_total = df['amount_in_gel'].sum()
    df.attrs['ytd_total'] = current_month_total + prev_month_amount
    df.attrs['current_month_total'] = current_month_total
    df.attrs['prev_month_total'] = prev_month_amount
    
    print(f"💰 Current month total: {current_month_total:,.2f} GEL")
    print(f"💰 Year-to-date total: {df.attrs['ytd_total']:,.2f} GEL")
    
    return df

# Load and process the data
df = load_and_process_tax_data('../data/synthetic_transactions.xlsx', prev_month_amount=2483205.00)

In [12]:
df.head(20)

,Transaction amount,Currency,Transaction date,Income source
0,50.0,GEL,2025-01-01,Bank transaction
1,60.5,EUR,2025-01-02,POS terminal payment
2,71.0,USD,2025-01-03,Cash
3,81.5,GEL,2025-01-04,"Payment system: PayPal, Wise, Deel, etc."
4,92.0,EUR,2025-01-05,Bank transaction
5,102.5,USD,2025-01-06,POS terminal payment
6,113.0,GEL,2025-01-07,Cash
7,123.5,EUR,2025-01-08,"Payment system: PayPal, Wise, Deel, etc."
8,134.0,USD,2025-01-09,Bank transaction
9,144.5,GEL,2025-01-10,POS terminal payment


In [ ]:
def get_currency_rate(currency: str, on_date: date) -> float:
    """
    Get currency exchange rate from Georgian National Bank API.
    
    Args:
        currency (str): Currency code (e.g., 'USD', 'EUR')
        on_date (date): Date for which to get the exchange rate
        
    Returns:
        float: Exchange rate to GEL
        
    Raises:
        Exception: If API request fails or currency not found
    """
    if currency == 'GEL':
        return 1.0
    
    try:
        address = f"https://nbg.gov.ge/gw/api/ct/monetarypolicy/currencies/en/json/?currencies={currency}&date={on_date}"
        response = get(address)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        data = response.json()
        if data and len(data) > 0 and 'currencies' in data[0] and len(data[0]['currencies']) > 0:
            rate = data[0]['currencies'][0]['rate']
            print(f"📈 {currency} rate on {on_date}: {rate}")
            return float(rate)
        else:
            raise ValueError(f"No rate data found for {currency} on {on_date}")
            
    except Exception as e:
        print(f"❌ Error getting rate for {currency} on {on_date}: {e}")
        raise

In [ ]:
def analyze_income_sources(df: pd.DataFrame) -> dict:
    """
    Analyze income by source from the processed tax data.
    
    Args:
        df (pd.DataFrame): Processed DataFrame with tax data
        
    Returns:
        dict: Dictionary with income amounts by source
    """
    income_sources = {
        'Cash': df[df['Income source'] == 'Cash']['amount_in_gel'].sum(),
        'POS Terminal': df[df['Income source'] == 'POS Terminal']['amount_in_gel'].sum(),
        'Bank Transfer': df[df['Income source'] == 'Bank Transfer']['amount_in_gel'].sum(),
        'Payment System': df[df['Income source'] == 'Payment system: PayPal, Wise, Deel, etc.']['amount_in_gel'].sum()
    }
    
    print("💼 Income by Source:")
    print("-" * 40)
    total = 0
    for source, amount in income_sources.items():
        print(f"{source:15}: {amount:>12,.2f} GEL")
        total += amount
    print("-" * 40)
    print(f"{'Total':15}: {total:>12,.2f} GEL")
    
    return income_sources

# Analyze income sources
income_analysis = analyze_income_sources(df)

In [ ]:
def generate_tax_summary(df: pd.DataFrame, income_sources: dict) -> dict:
    """
    Generate only the required tax report fields.
    Args:
        df (pd.DataFrame): Processed DataFrame with tax data
        income_sources (dict): Income breakdown by source
    Returns:
        dict: Only required fields for the report
    """
    summary = {
        'field_15_total_gel': df.attrs.get('ytd_total', 0),
        'field_18_cash_total': income_sources.get('Cash', 0),
        'field_19_pos_terminal_total': income_sources.get('POS Terminal', 0),
        'field_20_bank_transfer_total': income_sources.get('Bank Transfer', 0),
        'field_21_payment_system_total': income_sources.get('Payment System', 0)
    }
    print("\nTax Report Fields:")
    print(f"Field 15 (Total GEL from Jan 1 to today): {summary['field_15_total_gel']}")
    print(f"Field 18 (Cash total): {summary['field_18_cash_total']}")
    print(f"Field 19 (POS terminal payment total): {summary['field_19_pos_terminal_total']}")
    print(f"Field 20 (Bank transaction total): {summary['field_20_bank_transfer_total']}")
    print(f"Field 21 (Payment system total): {summary['field_21_payment_system_total']}")
    return summary

# Generate required tax summary fields only
tax_summary = generate_tax_summary(df, income_analysis)

In [ ]:
def export_tax_summary_to_excel(df: pd.DataFrame, summary: dict, output_path: str = '../data/tax_summary.xlsx') -> None:
    """
    Export tax summary and processed data to Excel file.
    
    Args:
        df (pd.DataFrame): Processed DataFrame with tax data
        summary (dict): Tax summary dictionary
        output_path (str): Path for the output Excel file
    """
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # Write processed transactions
        df.to_excel(writer, sheet_name='Processed_Transactions', index=False)
        
        # Write income source summary
        income_df = pd.DataFrame(list(summary['income_by_source'].items()), 
                                columns=['Income Source', 'Amount (GEL)'])
        income_df.to_excel(writer, sheet_name='Income_by_Source', index=False)
        
        # Write currency breakdown
        currency_df = pd.DataFrame(list(summary['currency_breakdown'].items()), 
                                  columns=['Currency', 'Amount (GEL)'])
        currency_df.to_excel(writer, sheet_name='Currency_Breakdown', index=False)
        
        # Write monthly breakdown
        monthly_df = pd.DataFrame(list(summary['monthly_breakdown'].items()), 
                                 columns=['Month', 'Amount (GEL)'])
        monthly_df.to_excel(writer, sheet_name='Monthly_Breakdown', index=False)
        
        # Write summary info
        summary_info = pd.DataFrame([
            ['Total Transactions', summary['period_info']['total_transactions']],
            ['Period Start', summary['period_info']['date_range']['start']],
            ['Period End', summary['period_info']['date_range']['end']],
            ['Current Period (GEL)', summary['amounts']['current_month_gel']],
            ['Previous Periods (GEL)', summary['amounts']['previous_months_gel']],
            ['Year-to-Date (GEL)', summary['amounts']['year_to_date_gel']]
        ], columns=['Metric', 'Value'])
        summary_info.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"📁 Tax summary exported to: {output_path}")

# Export the summary
# export_tax_summary_to_excel(df, tax_summary)

In [ ]:
def process_tax_data_workflow(
    input_file: str, 
    prev_month_amount: float = 0.0, 
    export_summary: bool = True,
    output_file: str = '../data/tax_summary.xlsx'
) -> tuple[pd.DataFrame, dict]:
    """
    Complete workflow for processing tax data from Excel file.
    Args:
        input_file (str): Path to input Excel file with tax data
        prev_month_amount (float): Previous month's amount in GEL
        export_summary (bool): Whether to export summary to Excel
        output_file (str): Path for output summary file
    Returns:
        tuple: (processed_dataframe, tax_report_fields_dict)
    """
    print("🚀 Starting Tax Data Processing Workflow")
    print("="*50)
    try:
        # Step 1: Load and process data
        df = load_and_process_tax_data(input_file, prev_month_amount=prev_month_amount)
        # Step 2: Analyze income sources
        income_analysis = analyze_income_sources(df)
        # Step 3: Generate required summary fields only
        summary = generate_tax_summary(df, income_analysis)
        # Step 4: Export if requested (optional, but summary now only has required fields)
        if export_summary:
            export_tax_summary_to_excel(df, summary, output_file)
        print("\n✅ Tax data processing completed successfully!")
        return df, summary
    except Exception as e:
        print(f"❌ Error in tax processing workflow: {e}")
        raise

# Example usage (commented out to prevent automatic execution)
# df_processed, summary_data = process_tax_data_workflow(
#     '../data/synthetic_transactions.xlsx', 
#     prev_month_amount=2483205.00
# )

In [ ]:
def validate_tax_data(df: pd.DataFrame) -> dict:
    """
    Validate tax data for common issues and provide data quality report.
    
    Args:
        df (pd.DataFrame): DataFrame with tax data to validate
        
    Returns:
        dict: Validation report with issues found
    """
    validation_report = {
        'total_rows': len(df),
        'issues': [],
        'warnings': [],
        'summary': {}
    }
    
    required_columns = ['Transaction amount', 'Currency', 'Transaction date', 'Income source']
    
    # Check for required columns
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        validation_report['issues'].append(f"Missing required columns: {missing_columns}")
    
    if len(missing_columns) == 0:
        # Check for missing values
        for col in required_columns:
            null_count = df[col].isnull().sum()
            if null_count > 0:
                validation_report['issues'].append(f"Column '{col}' has {null_count} missing values")
        
        # Check for negative amounts
        if 'Transaction amount' in df.columns:
            negative_amounts = (df['Transaction amount'] < 0).sum()
            if negative_amounts > 0:
                validation_report['warnings'].append(f"{negative_amounts} transactions have negative amounts")
        
        # Check for unusual currencies
        if 'Currency' in df.columns:
            unique_currencies = df['Currency'].unique()
            common_currencies = ['GEL', 'USD', 'EUR', 'GBP']
            unusual_currencies = [curr for curr in unique_currencies if curr not in common_currencies]
            if unusual_currencies:
                validation_report['warnings'].append(f"Unusual currencies found: {unusual_currencies}")
        
        # Check date range
        if 'Transaction date' in df.columns:
            try:
                dates = pd.to_datetime(df['Transaction date'])
                date_range = dates.max() - dates.min()
                validation_report['summary']['date_range_days'] = date_range.days
                
                # Check for future dates
                future_dates = (dates > pd.Timestamp.now()).sum()
                if future_dates > 0:
                    validation_report['warnings'].append(f"{future_dates} transactions have future dates")
            except:
                validation_report['issues'].append("Invalid date format in 'Transaction date' column")
    
    # Print validation report
    print("🔍 DATA VALIDATION REPORT")
    print("="*40)
    print(f"📊 Total rows: {validation_report['total_rows']}")
    
    if validation_report['issues']:
        print("\n❌ ISSUES FOUND:")
        for issue in validation_report['issues']:
            print(f"  • {issue}")
    
    if validation_report['warnings']:
        print("\n⚠️  WARNINGS:")
        for warning in validation_report['warnings']:
            print(f"  • {warning}")
    
    if not validation_report['issues'] and not validation_report['warnings']:
        print("\n✅ No issues found - data looks good!")
    
    return validation_report

# Validate the current data
validation_result = validate_tax_data(df)

In [ ]:
def filter_transactions_by_date(df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Filter transactions by date range.
    
    Args:
        df (pd.DataFrame): DataFrame with transaction data
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        
    Returns:
        pd.DataFrame: Filtered DataFrame
    """
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    
    mask = (pd.to_datetime(df['Transaction date']) >= start) & (pd.to_datetime(df['Transaction date']) <= end)
    filtered_df = df[mask].copy()
    
    print(f"📅 Filtered {len(filtered_df)} transactions between {start_date} and {end_date}")
    return filtered_df

def filter_transactions_by_source(df: pd.DataFrame, income_sources: list) -> pd.DataFrame:
    """
    Filter transactions by income source.
    
    Args:
        df (pd.DataFrame): DataFrame with transaction data
        income_sources (list): List of income sources to include
        
    Returns:
        pd.DataFrame: Filtered DataFrame
    """
    filtered_df = df[df['Income source'].isin(income_sources)].copy()
    print(f"💼 Filtered {len(filtered_df)} transactions from sources: {income_sources}")
    return filtered_df

def get_monthly_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Get monthly summary of transactions.
    
    Args:
        df (pd.DataFrame): DataFrame with transaction data
        
    Returns:
        pd.DataFrame: Monthly summary
    """
    df_copy = df.copy()
    df_copy['month'] = pd.to_datetime(df_copy['Transaction date']).dt.to_period('M')
    
    monthly_summary = df_copy.groupby('month').agg({
        'Transaction amount': 'sum',
        'amount_in_gel': 'sum',
        'Currency': 'count'
    }).rename(columns={'Currency': 'transaction_count'})
    
    print("📊 Monthly Summary:")
    print(monthly_summary)
    return monthly_summary

# Example usage (commented)
# monthly_data = get_monthly_summary(df)

In [ ]:
# 📋 Tax Data Processing Functions - Usage Examples

This notebook now contains a complete set of functions for processing tax data. Here's how to use them:

## 🚀 Quick Start

```python
# 1. Create a template file
create_tax_template('../data/my_template.xlsx')

# 2. Process your tax data (after filling the template)
df, summary = process_tax_data_workflow(
    '../data/my_filled_data.xlsx', 
    prev_month_amount=1000000.00,  # Previous period amount in GEL
    export_summary=True
)

# 3. Validate your data
validation = validate_tax_data(df)
```

## 🔧 Advanced Usage

```python
# Filter data by date range
q1_data = filter_transactions_by_date(df, '2025-01-01', '2025-03-31')

# Filter by income source
cash_only = filter_transactions_by_source(df, ['Cash'])

# Get monthly breakdown
monthly_summary = get_monthly_summary(df)

# Get specific currency rate
rate = get_currency_rate('USD', date(2025, 9, 10))
```

In [ ]:
# Display current processed data
print("📊 Current Data Overview:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst 5 rows:")
df.head()

In [36]:
field_study = pd.read_excel('/Users/aershova/vscode-projects/nlp_toolkit/clustering/data/Налоговый хелпер.xlsx', sheet_name='Sheet1')

In [37]:
field_study

,Сумма поступления,Валюта,Дата поступления (для иностранных валют),Вид поступления,Ссылка на нацбанк,джсон с курсом,Курс,Сумма
0,1993,EUR,22.08.2025,на счёт,https://nbg.gov.ge/gw/api/ct/monetarypolicy/cu...,"[{""date"":""2025-08-22T00:00:00.000Z"",""currencie...",3.1532,6284.3276
1,222.6,GEL,01.08.2025,"через платёжную систему (PayPal, Wise, Payonee...",NaN,NaN,1.0000,222.6000
2,150.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Итого:,NaN,NaN,NaN,NaN,NaN,NaN,6506.9276
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
field_study = field_study[0:10]

In [ ]:
# Output required fields for tax report
print("Field 15 (Total GEL from Jan 1 to today):", summary['amounts']['year_to_date_gel'])
print("Field 18 (Cash total):", summary['income_by_source']['Cash'])
print("Field 19 (POS terminal payment total):", summary['income_by_source']['POS Terminal'])
print("Field 20 (Bank transaction total):", summary['income_by_source']['Bank Transfer'])
print("Field 21 (Payment system total):", summary['income_by_source']['Payment System'])